# Instalação de pacotes e configurações

In [1]:
%pip install geopandas
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install descartes 
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Importando o arquivo e suas páginas.


In [3]:
# Importa o arquivo Excel com suas páginas VENDAS e PRODUTOS
frame_df = pd.read_excel('Base-Dados-Desafio-D&A-01.xlsx',sheet_name=['VENDAS','PRODUTOS'])
sales_df = frame_df.get('VENDAS')
products_df = frame_df.get('PRODUTOS')

# Mostra a tabela principal com suas páginas 
frame_df

{'VENDAS':     CLIENTE  IDADE             ESTADO PRODUTO  QUANTIDADE_VENDIDA       DATA
 0         1     47     SANTA CATARINA       A                  64 2021-01-05
 1         1     47     SANTA CATARINA       B                 488 2021-01-15
 2         1     47     SANTA CATARINA       C                 420 2021-01-25
 3         1     47     SANTA CATARINA       D                 259 2021-02-04
 4         1     47     SANTA CATARINA       E                 387 2021-02-14
 5         2     61          SÃO PAULO       A                 302 2021-02-24
 6         2     61          SÃO PAULO       B                 117 2021-03-06
 7         2     61          SÃO PAULO       C                 344 2021-03-16
 8         2     61          SÃO PAULO       D                 292 2021-04-15
 9         2     61          SÃO PAULO       E                 447 2021-04-25
 10        3     26              BAHIA       A                  42 2021-05-05
 11        3     26              BAHIA       B        

# Preparação do DataSet

In [ ]:
# Convertendo a coluna 'DATA' para o formato de data
sales_df['DATA'] = pd.to_datetime(sales_df['DATA'], infer_datetime_format=True)

# Coletando o mês da coluna 'DATA'
sales_df['Mes'] = sales_df['DATA'].dt.month  # Para evitar sobre-escrever, a coluna 'Mes' foi criada

# Importando, tratando e criando tabelas para a demografia.

In [ ]:
# Filtrando a tabela em uma tabela com os Estados e quantas vezes foram repitidos.

df = frame_df['VENDAS']
df = df['ESTADO'] 

counts = df.value_counts() # Conta quantas vezes o Estado aparece.

# Cria um novo Data Frame com os Estados e sua ocorrência.
df_demographics = pd.DataFrame({
    'Estados': counts.index,
    'Contagem': counts.values
})

# Converte os Estados em suas respectivas UF's para assim ser possivel o merge com a tabela do IBGE.
df_demographics['Estados'] = df_demographics['Estados'].replace({
    'SANTA CATARINA': 'SC',
    'PARAÍBA': 'PB',
    'SÃO PAULO': 'SP',
    'BAHIA': 'BA',
    'RIO GRANDE DO SUL': 'RS',
    'RIO DE JANEIRO': 'RJ',
    'PARANÁ': 'PR',
})
df_demographics

In [ ]:
# Lê o arquivo do IBGE e seleciona a camada das UF's
ibge_table = gpd.read_file('bcim_2016_21_11_2018.gpkg', layer='lim_unidade_federacao_a')

# Altera o nome da coluna para possibilitar a mesclagem
ibge_table.rename({'sigla': 'Estados'}, axis=1, inplace=True)

# Mescla as duas tabelas com base na coluna 'Estados'
Brasil = ibge_table.merge(df_demographics, on='Estados', how='outer')

# Seleciona as colunas 'Estados', 'Contagem' e 'geometry' do DataFrame 'Brasil'
Brasil[['Estados', 'Contagem', 'geometry']]

# Altera os valores NaN na coluna 'Contagem' para 0
Brasil['Contagem'] = Brasil['Contagem'].fillna(0)

---
# *Representação gráfica da demografia*.

In [ ]:
%matplotlib inline

Brasil.plot(column = 'Contagem',cmap =  'Reds', figsize= (15,9), legend= True, edgecolor= 'gray') # Montagem do gráfico.


Esse gráfico representa a compra de produtos por Estado.A distribuição no Brasil por região: 

* Região Sul = 15
    * Santa Catarina = 7
    * Rio Grande do Sul = 5
    * Paraná = 2
* Região Nordeste = 13
    * Paraíba = 8
    * Bahia = 5
* Região Sudeste = 9
    * São Paulo = 5
    * Rio de Janeiro = 4

Sendo assim, é notável que a região com a maior densidade de consumidores é a região sul do país e sendo em potencial o melhor publico. O mercado no nordeste não é fraco e poderia ser expandido já que os produtos são bem recebidos na Paraíba e Bahia. Já no sudeste o mercado pode se popularizar pela alta densidade que ambos estados possuem e um mercado bem movimentado economicamente.